## Demonstration of the LON algorithm

In [1]:
import numpy as np
from scipy import optimize
from functions import AckleyMin3d
beta = 0.4546
multiplier = 2
# random seeding
seed =12345
np.random.seed(seed)

In [2]:
# basin hopping
def basin_hopping(problem, step_size, 
                    iter_limit = 1000, opt_config={}):
    # x_tolerance up to 5 decimals
    function = problem.fitness
    lower_bounds, upper_bounds = problem.get_bounds()
    optima = []
    values = []
    # local search parameters -- from LON paper
    method = opt_config.get("method", "L-BFGS-B")
    tol = opt_config.get("tol", 10**-7)
    maxiter = opt_config.get("maxiter", 15000)
    # initial solution
    x_init = np.random.uniform(low=lower_bounds, high=upper_bounds)
    res = optimize.minimize(function, x0=x_init, method=method, tol=tol,
                            options={"maxiter":maxiter})
    if res["success"]:
        x_opt = res["x"]
        optima.append(x_opt)
        values.append(function(x_opt))
    else:
        raise Exception("Optimiser failed!")
    counter = 0
    while True:
        x_new = x_opt + np.random.uniform(low=-step_size, high=step_size)
        x_new = np.clip(x_new, lower_bounds, upper_bounds)
        res = optimize.minimize(function, x0=x_new, method=method, tol=tol,
                                    options={"maxiter":maxiter})
        if res["success"]:
            x_nopt = res["x"]
        else:
            raise Exception("Optimiser failed!")
        if function(x_nopt) <= function(x_opt):
            x_opt = x_nopt
            optima.append(x_opt)
            values.append(function(x_opt))
            counter = 0
        else:
            counter += 1
        if counter >= iter_limit:
            break
    return np.array(optima), np.array(values)

In [3]:
optima, values = basin_hopping(AckleyMin3d, beta*multiplier)

In [4]:
optima, values

(array([[ 28.50489636, -12.50206497, -20.50349945],
        [ 27.50463797, -13.50228832, -21.50362842],
        [ 27.50465012, -13.5022826 , -21.50363543],
        [ 26.50433106, -14.50236963, -22.50367722],
        [ 25.50397988, -15.50242599, -23.50367606],
        [ 25.50398043, -15.5024254 , -23.50367547],
        [ 25.50398257, -15.50242246, -23.5036724 ],
        [ 25.50398423, -15.50242164, -23.50367172],
        [ 25.50398428, -15.50242168, -23.50367169],
        [ 24.50364754, -16.50241644, -24.50360069],
        [ 23.50325169, -17.50242141, -25.50352849],
        [ 22.50288638, -18.50237321, -26.50339964],
        [ 21.50253326, -19.50229765, -27.50324033],
        [ 20.50219781, -20.5021996 , -28.50305769],
        [ 19.50188835, -21.50208211, -29.50285694],
        [ 19.50188855, -21.50208226, -29.50285708],
        [ 19.50188846, -21.5020822 , -29.502857  ],
        [ 18.50160493, -22.50195181, -30.50264584],
        [ 18.50160487, -22.50195195, -30.50264603],
        [ 17

In [5]:
best_ind = np.argmin(values)
optima[best_ind], values[best_ind]

(array([ 23.50039692, -33.50056659, -35.50060012]), -22.312003172722264)